In [1]:
import sys
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

sys.path.append('..')
from audio_toolbox.metrics import splitTabularPredicting

audio_root_folder = '../archive/Data'
labels_csv = os.path.join(audio_root_folder, 'data.csv')
unslicedData = pd.read_csv(labels_csv, header=0)
slicedCSV = os.path.join(audio_root_folder, 'features.csv')
slicedDataSet = pd.read_csv(slicedCSV, header=0)
label = unslicedData['label']
unslicedData = unslicedData.sample(frac=1).reset_index(drop=True)

In [2]:
#generate the testing data folds
K = 5
testDataFolds = []
length = len(unslicedData)
slice_size = length // K
start_index = 0
for i in range(K):
    end_index = min(start_index + slice_size, length)
    fold = unslicedData.iloc[start_index: end_index]
    start_index = end_index
    testDataFolds.append(fold)

In [3]:
#generate the training data folds
trainData = []
for i in range(K):
    index = testDataFolds[i]['index']
    trainData.append(unslicedData[~unslicedData['index'].isin(index)])

In [4]:
#model training
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#model = RandomForestClassifier(max_depth = 15, max_features = 5)
model = LogisticRegression(max_iter=1000, C=1e-2)
predictions = []
for i in range(K):
    X_train = trainData[i]
    label = X_train['label']
    X_train.drop(['label','index'], axis=1, inplace=True)
    model.fit(X_train, label)
    trainAccuracy = accuracy_score(model.predict(X_train), label)
    print(f"training accuracy: {trainAccuracy}")
    prediction= splitTabularPredicting(model, testDataFolds[i], slicedDataSet)
    predictions.append(prediction)

C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_31716\3762941563.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_31716\3762941563.py:12: SettingWithCopyWarni

training accuracy: 0.40625


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_31716\3762941563.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.3675


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_31716\3762941563.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.39875


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Hao Hu\AppData\Local\Temp\ipykernel_31716\3762941563.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(['label','index'], axis=1, inplace=True)


training accuracy: 0.4025
training accuracy: 0.37625


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
#average accuracy
accuracies = []
for i in range(K):
    testLabel = testDataFolds[i]['label']
    prediction = predictions[i]
    accuracy = accuracy_score(testLabel, prediction)
    accuracies.append(accuracy)

mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")

Mean Accuracy: 0.32160804020100503
